# 03_graph
* process news SPO analytics via graph algorithm

### install extra library

In [1]:
!pip install networkx

### read news SPO from DataPlatform

In [2]:
from dsmlibrary.datanode import DataNode
from tqdm.auto import tqdm

In [3]:
datanode = DataNode()

Init DataNode sucessful!


In [8]:
dir_process_id = 44

In [9]:
df = datanode.read_df(datanode.get_file_id(directory_id=dir_process_id, name="Analysis_News_1.parquet"))
df

,subject,predicate,object
0,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,ชุมชนโรงหวาย
1,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,โรงหวาย
2,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,หวาย
3,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,วาย
4,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,PER,สวนหลวง ศานนท์ หวังสร้างบุญ
...,...,...,...
257,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,ภาคทางการศึกษา
258,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,ทางการศึกษา
259,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,การศึกษา
260,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,THE STANDARD


### prepair graph data

In [10]:
graph_data = [(elm.get('subject'), elm.get('object')) for elm in df.to_dict('records')]
graph_data

[('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'ชุมชนโรงหวาย'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'โรงหวาย'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'หวาย'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'วาย'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'สวนหลวง ศานนท์ หวังสร้างบุญ'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'หลวง ศานนท์ หวังสร้างบุญ'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  ' ศานนท์ หวังสร้างบุญ'),
 ('กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'ศานนท์ หวังสร้างบุญ'),
 ('กทม. ค้นห

In [11]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [12]:
output_notebook()

Loading BokehJS ...

In [ ]:
'''
use for clear bokeh plot
'''
# from bokeh.io import curdoc
# curdoc().clear()

In [13]:
news_titles = {elm:'name' for elm in list(df['subject'].unique())}
news_titles

{'กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน': 'name',
 'เด็กทุกคนต้องได้เรียน! ‘แสนสิริ’ ออกหุ้นกู้ 100 ล้านบาท สนับสนุนกองทุนเพื่อความเสมอภาคทางการศึกษา นำร่องที่ จ.ราชบุรี ตั้งเป้าเด็กหลุดจากระบบการศึกษาเป็น ‘ศูนย์’ ภายใน 3 ปี': 'name',
 'เจาะข้อมูล ‘เด็กหลุดจากระบบการศึกษา’ เมื่อป้ายผ้าและการแขวนคอสะท้อนปัญหาที่รุมเร้าเด็กไทย': 'name',
 'ศูนย์นโยบายไทยสร้างไทย จี้รัฐบาลแก้ปัญหาสินค้าเกษตรตกต่ำ ลามเป็นปัญหาครอบครัว กระทบระบบการศึกษา ทำเด็กหลุดออกจากการเรียนหนังสือ': 'name',
 'ชัชชาติ ลงพื้นที่เขตราษฎร์บูรณะ-ทุ่งครุ เปิดแผนช่วยคนรายได้น้อยในกรุงเทพฯ เข้าถึงบ้านมั่นคง พร้อมย้ำนโยบายพัฒนากรุงเทพฯ 9 มิติ': 'name',
 '‘กลุ่มนักเรียนเลว’ แสดงออกเชิงสัญลักษณ์วันครู มอบพานไว้อาลัยระบบการศึกษาให้ตรีนุช เรียกร้องแก้ไขปัญหาให้ตรงจุด': 'name',
 'กสศ. เปิดผลการดำเนินงาน 2564 พบปัญหาโควิดซ้ำเติมความเหลื่อมล้ำ ทำเด็กยากจนหลุดระบบการศึกษา': 'name',
 'มองปัญหาความเหลื่อมล้ำไทยในมุมนักธุรกิจและผู้เชี่ยวชาญ ชี้ความร่วมมือทุกภาคส่วน การแข่งขันที่เป็น

In [16]:
G = nx.Graph(graph_data)
G

In [17]:
node_colors = {elm:'skyblue' if elm in news_titles else 'yellow' for elm in G.nodes()}
node_colors

{'กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน': 'skyblue',
 'ชุมชนโรงหวาย': 'yellow',
 'โรงหวาย': 'yellow',
 'หวาย': 'yellow',
 'วาย': 'yellow',
 'สวนหลวง ศานนท์ หวังสร้างบุญ': 'yellow',
 'หลวง ศานนท์ หวังสร้างบุญ': 'yellow',
 ' ศานนท์ หวังสร้างบุญ': 'yellow',
 'ศานนท์ หวังสร้างบุญ': 'yellow',
 'านนท์ หวังสร้างบุญ': 'yellow',
 ' หวังสร้างบุญ': 'yellow',
 'หวังสร้างบุญ': 'yellow',
 'สร้างบุญ': 'yellow',
 'บุญ': 'yellow',
 'กรุงเทพมหานคร': 'yellow',
 ' ไกรยส ภัทราวาท': 'yellow',
 'ไกรยส ภัทราวาท': 'yellow',
 'กรยส ภัทราวาท': 'yellow',
 'ยส ภัทราวาท': 'yellow',
 'ส ภัทราวาท': 'yellow',
 ' ภัทราวาท': 'yellow',
 'ภัทราวาท': 'yellow',
 'าวาท': 'yellow',
 'วาท': 'yellow',
 'ท': 'yellow',
 'กองทุนเพื่อความเสมอภาคทางการศึกษา': 'yellow',
 'เพื่อความเสมอภาคทางการศึกษา': 'yellow',
 'ความเสมอภาคทางการศึกษา': 'yellow',
 'เสมอภาคทางการศึกษา': 'yellow',
 'ภาคทางการศึกษา': 'yellow',
 'ทางการศึกษา': 'yellow',
 'การศึกษา': 'yellow',
 'กสศ.': 'yellow',


In [ ]:
# plt.figure(figsize=(16,8))
# nx.draw_kamada_kawai(G, node_size=100)
# plt.show()

In [ ]:
# plt.figure(figsize=(16,8))
# nx.draw_spring(G, node_size=100)
# plt.show()

### visualize raw data

In [18]:
title = "News and NameEntities"
HOVER_TOOLTIPS = [("Character", "@index")]
nx.set_node_attributes(G, node_colors, 'node_colors')
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset,hover", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title,
              frame_width=1200, frame_height=800
 )

network_graph = from_networkx(G, nx.kamada_kawai_layout, scale=10, center=(0, 0))
network_graph.node_renderer.glyph = Circle(size=15, fill_color='node_colors')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)

### Compute betweenness_centrality graph


In [19]:
def cal_size(val, k=10, k2=1000):
    val = val * k * k2
    if val > k: 
        return val
    return k

In [20]:
graph_values = nx.centrality.betweenness.betweenness_centrality(G)
node_size = {k:cal_size(v, k=10, k2=1000) for k,v in graph_values.items()}

In [21]:
title = "betweenness_centrality"
HOVER_TOOLTIPS = [("Character", "@index")]

nx.set_node_attributes(G, node_size, 'node_size')
nx.set_node_attributes(G, node_colors, 'node_colors')

plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset,hover", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title,
              frame_width=1200, frame_height=800
)

network_graph = from_networkx(G, nx.kamada_kawai_layout, scale=10, center=(0, 0))
network_graph.node_renderer.glyph = Circle(fill_color='node_colors', size='node_size')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)

### compute eigenvector_centrality

In [22]:
graph_values = nx.centrality.eigenvector.eigenvector_centrality(G, max_iter=600)
node_size = {k:cal_size(v, k=10, k2=15) for k,v in graph_values.items()}

In [23]:
title = "xxx"
HOVER_TOOLTIPS = [("Character", "@index")]

nx.set_node_attributes(G, node_size, 'node_size') 
nx.set_node_attributes(G, node_colors, 'node_colors')

plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset,hover", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title,
              frame_width=1200, frame_height=800
)

network_graph = from_networkx(G, nx.kamada_kawai_layout, scale=10, center=(0, 0))
network_graph.node_renderer.glyph = Circle(fill_color='node_colors', size='node_size')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)